### dataframe 형태로 되어있는 데이터를 numpy 배열로 만들어서 뽑아내는 코드
- 딥러닝(특히 LSTM) 돌릴 때 사용하기 위함임
- macro data는 그냥 numpy 배열로만 만들면 되는데 
- firm-specific data는 데이터가 존재하지 않는 기간에 대한 row를 만들어서 그 값을 -99.99로 채우는 작업 필요

- macro [dataframe] -> macro [numpy]
- firm-specific [dataframe] -> firm-specific [numpy]

- firm-specific + macro [dataframe] -> firm-specific + macro [numpy]
- firm-specific + macro hidden states [dataframe] -> firm-specific + macro hidden states [numpy]




# macro : [dataframe] -> [numpy]

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 데이터프레임으로 되어있는 macro 데이터를 넘파이 배열로 변환 후 npz 파일로 저장하는 함수
def macro_to_npz(d_mac, part):
    mac_date = d_mac['sasdate'].astype(str).to_numpy()
    mac_variables = d_mac.columns[1:].to_numpy()
    d_mac_np = d_mac.iloc[:,1:].to_numpy()
    np.savez(f'my_macro_{part}', date=mac_date, variable=mac_variables, data=d_mac_np)

In [3]:
# 불러오기 전에 엑셀파일에서 두번째 행(Transform:) 삭제하기
mac = pd.read_csv('./data/fred_macro_2022-07.csv')
mac['sasdate'] = pd.to_datetime(mac['sasdate'])

- macro 데이터도 firm-specific 데이터와 마찬가지로 변수마다 업데이트되는 주기가 다름
- 어떤 변수는 3달 후에 알 수 있고(lag3_vars), 어떤 변수는 2달 후에 알 수 있고(lag2_vars) 등등..
- 이런 변수별 업데이트 주기를 반영해야 함 
- 이때, 현재 시점에서 알 수 있는 가장 최근 값을 사용하도록 함

In [4]:
lag3_vars = ['CMRMTSPLx', 'HWI', 'HWIURATIO', 'ACOGNO', 'BUSINVx', 'ISRATIOx', 'NONREVSL', 'CONSPI', 
             'S&P div yield', 'DTCOLNVHFNM', 'DTCTHFNM']
lag5_vars = ['S&P PE ratio']
lag2_vars = mac.columns.drop(['sasdate']+lag3_vars+lag5_vars).tolist()

# 변수별로 업데이트 주기를 반영해서 lag 시키는 함수
def lag_vars(macro_data):
    macro_data[lag2_vars] = macro_data[lag2_vars].shift(2).bfill()
    macro_data[lag3_vars] = macro_data[lag3_vars].shift(3).bfill()
    macro_data[lag5_vars] = macro_data[lag5_vars].shift(5).bfill()

In [5]:
lag_vars(mac)

In [6]:
mac

,sasdate,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
0,1959-01-01,2442.158,2293.2,17.272,2.922664e+05,18235.77392,22.0151,23.3984,22.2848,31.5847,...,17.791,11.326,2.13,2.45,2.04,95.3,6476.00,12298.00,84.2043,19.5715
1,1959-02-01,2442.158,2293.2,17.272,2.922664e+05,18235.77392,22.0151,23.3984,22.2848,31.5847,...,17.791,11.326,2.13,2.45,2.04,95.3,6476.00,12298.00,84.2043,19.5715
2,1959-03-01,2442.158,2293.2,17.272,2.922664e+05,18235.77392,22.0151,23.3984,22.2848,31.5847,...,17.791,11.326,2.13,2.45,2.04,95.3,6476.00,12298.00,84.2043,19.5715
3,1959-04-01,2451.778,2301.5,17.452,2.922664e+05,18369.56308,22.4463,23.7142,22.4778,31.8164,...,17.798,11.343,2.14,2.46,2.05,95.3,6476.00,12298.00,83.5280,19.5715
4,1959-05-01,2467.594,2318.5,17.617,2.944247e+05,18523.05762,22.7696,23.8577,22.5882,31.8164,...,17.785,11.363,2.15,2.45,2.07,95.3,6476.00,12298.00,81.6405,19.5715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,2022-02-01,17791.488,14465.3,123.868,1.572476e+06,634393.00000,101.7587,101.6136,101.8486,102.6412,...,106.351,126.834,27.17,31.25,24.38,70.6,369983.50,754662.69,5672.6883,21.2985
758,2022-03-01,17704.448,14441.0,125.521,1.562935e+06,651557.00000,102.1460,102.3739,102.8209,104.2414,...,107.157,127.273,27.31,31.44,24.51,67.2,370554.92,753781.25,5766.5772,22.9143
759,2022-04-01,17724.106,14484.3,125.552,1.581756e+06,662321.00000,102.9981,103.4073,103.6360,104.4925,...,109.082,127.593,27.42,31.60,24.56,62.8,370286.86,751635.23,5810.8647,26.1429
760,2022-05-01,17665.439,14442.9,125.880,1.569743e+06,669958.00000,103.7286,103.7781,104.1434,104.7319,...,112.165,128.206,27.53,31.72,24.74,59.4,374193.90,753730.65,5818.0494,26.9368


In [7]:
def make_stationary(macro_data):
    d_mac = macro_data.copy()
    for col in d_mac.iloc[:,1:].columns:
        d_mac[f'd_{col}'] = (d_mac[col] - d_mac[col].shift(1).bfill())/d_mac[col]
        d_mac.drop(col, axis=1, inplace=True)
    return d_mac

In [8]:
# 비정상계열인 데이터를 정상계열로 만들어서 쓰기 위해 차분 후 나눔 (return 계산하듯이)
# make_stationary 함수 잃어버림..다시 짜야 해 
d_mac = make_stationary(mac)
d_mac.head()

,sasdate,d_RPI,d_W875RX1,d_DPCERA3M086SBEA,d_CMRMTSPLx,d_RETAILx,d_INDPRO,d_IPFPNSS,d_IPFINAL,d_IPCONGD,...,d_DNDGRG3M086SBEA,d_DSERRG3M086SBEA,d_CES0600000008,d_CES2000000008,d_CES3000000008,d_UMCSENTx,d_DTCOLNVHFNM,d_DTCTHFNM,d_INVEST,d_VIXCLSx
0,1959-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
1,1959-02-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
2,1959-03-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
3,1959-04-01,0.003924,0.003606,0.010314,0.000000,0.007283,0.019210,0.013317,0.008586,0.007282,...,0.000393,0.001499,0.004673,0.004065,0.004878,0.0,0.0,0.0,-0.008097,0.0
4,1959-05-01,0.006409,0.007332,0.009366,0.007331,0.008287,0.014199,0.006015,0.004888,0.000000,...,-0.000731,0.001760,0.004651,-0.004082,0.009662,0.0,0.0,0.0,-0.023120,0.0


In [9]:
def macro_clean_data(d_mac_data):
    for col in d_mac_data.iloc[:,1:].columns:
        is_inf = (d_mac_data[col] == float("inf")) | (d_mac_data[col] == float("-inf"))
        res = sum(is_inf)
        if res != 0:
            d_mac_data.loc[is_inf,col] = 0

In [10]:
macro_clean_data(mac)

In [11]:
mac_all = mac[(mac['sasdate'] >= '1996-01-01') & (mac['sasdate'] <= '2021-12-31')]

In [12]:
macro_clean_data(d_mac)

In [13]:
d_mac.head()

,sasdate,d_RPI,d_W875RX1,d_DPCERA3M086SBEA,d_CMRMTSPLx,d_RETAILx,d_INDPRO,d_IPFPNSS,d_IPFINAL,d_IPCONGD,...,d_DNDGRG3M086SBEA,d_DSERRG3M086SBEA,d_CES0600000008,d_CES2000000008,d_CES3000000008,d_UMCSENTx,d_DTCOLNVHFNM,d_DTCTHFNM,d_INVEST,d_VIXCLSx
0,1959-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
1,1959-02-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
2,1959-03-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
3,1959-04-01,0.003924,0.003606,0.010314,0.000000,0.007283,0.019210,0.013317,0.008586,0.007282,...,0.000393,0.001499,0.004673,0.004065,0.004878,0.0,0.0,0.0,-0.008097,0.0
4,1959-05-01,0.006409,0.007332,0.009366,0.007331,0.008287,0.014199,0.006015,0.004888,0.000000,...,-0.000731,0.001760,0.004651,-0.004082,0.009662,0.0,0.0,0.0,-0.023120,0.0


In [14]:
# train test split
d_mac_all = d_mac[(d_mac['sasdate'] >= '1996-01-01') & (d_mac['sasdate'] <= '2021-12-31')]
d_mac_train = d_mac[(d_mac['sasdate'] >= '1996-01-01') & (d_mac['sasdate'] <= '2011-12-31')]
d_mac_valid = d_mac[(d_mac['sasdate'] >= '2012-01-01') & (d_mac['sasdate'] <= '2016-12-31')]
d_mac_test = d_mac[(d_mac['sasdate'] >= '2017-01-01') & (d_mac['sasdate'] <= '2021-12-31')]

In [15]:
print(d_mac_all.shape)
print(d_mac_train.shape)
print(d_mac_valid.shape)
print(d_mac_test.shape)

(312, 128)
(192, 128)
(60, 128)
(60, 128)


In [16]:
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaler.fit(d_mac_all.iloc[:,1:])
# d_mac_all.iloc[:,1:] = scaler.transform(d_mac_all.iloc[:,1:])

In [17]:
# npz 파일로 저장
macro_to_npz(d_mac_all, 'all')
macro_to_npz(d_mac_train, 'train')
macro_to_npz(d_mac_valid, 'valid')
macro_to_npz(d_mac_test, 'test')

In [18]:
# npz 파일 불러오기
# my_macro_all = np.load('my_macro_all.npz', allow_pickle=True)

# my_macro_train = np.load('my_macro_train.npz', allow_pickle=True)
# my_macro_valid = np.load('my_macro_valid.npz', allow_pickle=True)
# my_macro_test = np.load('my_macro_test.npz', allow_pickle=True)